In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import gamma, norm

from functools import reduce

from gproc.generative import sample_at_x
from gproc.kernels import *

In [8]:
# Data sizes and dimension
D = 2
N = 50
M = 50

x_1 = np.random.uniform(-1, 1, N * D).reshape(-1, D) # Reshape to N x D matrix
x_2 = np.random.uniform(-1, 1, M * D).reshape(-1, D) # Reshape to N x D matrix

In [15]:
Kernel = SquaredExponential(lengthscale = 0.4, variance = 0.5)
Kernel.make_gram(x_1, x_2)

array([[0.01109761, 0.02844934, 0.01708996, ..., 0.0170425 , 0.22316104,
        0.34114449],
       [0.09540421, 0.11163667, 0.10460524, ..., 0.11207184, 0.20599508,
        0.26189349],
       [0.04052111, 0.04684166, 0.04067562, ..., 0.06777993, 0.16726198,
        0.39323648],
       ...,
       [0.00411736, 0.09475691, 0.0288173 , ..., 0.00118114, 0.41423739,
        0.01601829],
       [0.03811173, 0.34209404, 0.20910285, ..., 0.00390382, 0.0680986 ,
        0.00062117],
       [0.01674812, 0.21081006, 0.08608402, ..., 0.00479441, 0.44683407,
        0.02133025]])